# 第4回講義 宿題

## 課題

今Lessonで学んだことを元に，MNISTのファッション版 (Fashion MNIST，クラス数10) を多層パーセプトロンによって分類してみましょう．

Fashion MNISTの詳細については以下のリンクを参考にしてください．

Fashion MNIST: https://github.com/zalandoresearch/fashion-mnist

### 目標値

Accuracy 85%

### ルール

- 訓練データはx_train，t_train，テストデータはx_testで与えられます．
- 予測ラベルはone_hot表現ではなく0~9のクラスラベルで表してください．
- **下のセルで指定されているx_train、t_train以外の学習データは使わないでください**．
- Pytorchを利用して構いません．
- ただし，**torch.nn.Conv2dのような高レベルのAPIは使用しないで下さい**．具体的には，nn.Parameter, nn.Module, nn.Sequential以外のnn系のAPIです．
- torchvision等で既に実装されているモデルも使用しないで下さい．

### 提出方法

- 2つのファイルを提出していただきます．
  - テストデータ (x_test) に対する予測ラベルをcsvファイル (ファイル名: submission_pred.csv) で提出してください．
  - それに対応するpythonのコードをsubmission_code.pyとして提出してください (%%writefileコマンドなどを利用してください)．

### 評価方法

- 予測ラベルのt_testに対する精度 (Accuracy) で評価します．
- 即時採点しLeader Boardを更新します．（採点スケジュールは別アナウンス）
- 締切後の点数を最終的な評価とします．

### ドライブのマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### データの読み込み（この部分は修正しないでください）

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import inspect
 
nn_except = ["Module", "Parameter", "Sequential"]
for m in inspect.getmembers(nn):
    if not m[0] in nn_except and m[0][0:2] != "__":
        delattr(nn, m[0]) 

seed = 1234
torch.manual_seed(seed)
np.random.seed(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#学習データ
x_train = np.load('drive/MyDrive/Colab Notebooks/DLBasics2023_colab/Lecture04/data/x_train.npy')
t_train = np.load('drive/MyDrive/Colab Notebooks/DLBasics2023_colab/Lecture04/data/y_train.npy')
    
#テストデータ
x_test = np.load('drive/MyDrive/Colab Notebooks/DLBasics2023_colab/Lecture04/data/x_test.npy')

class train_dataset(torch.utils.data.Dataset):
    def __init__(self, x_train, t_train):
        self.x_train = x_train.reshape(-1, 784).astype('float32') / 255
        self.t_train = t_train

    def __len__(self):
        return self.x_train.shape[0]

    def __getitem__(self, idx):
        return torch.tensor(self.x_train[idx], dtype=torch.float), torch.tensor(self.t_train[idx], dtype=torch.long)

class test_dataset(torch.utils.data.Dataset):
    def __init__(self, x_test):
        self.x_test = x_test.reshape(-1, 784).astype('float32') / 255

    def __len__(self):
        return self.x_test.shape[0]

    def __getitem__(self, idx):
        return torch.tensor(self.x_test[idx], dtype=torch.float)

trainval_data = train_dataset(x_train, t_train)
test_data = test_dataset(x_test)

### 多層パーセプトロンの実装

In [ ]:
batch_size = 32

val_size = 10000
train_size = len(trainval_data) - val_size

train_data, val_data = torch.utils.data.random_split(trainval_data, [train_size, val_size])

dataloader_train = torch.utils.data.DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=True
)

dataloader_valid = torch.utils.data.DataLoader(
    val_data,
    batch_size=batch_size,
    shuffle=True
)

dataloader_test = torch.utils.data.DataLoader(
    test_data,
    batch_size=batch_size,
    shuffle=False
)

In [ ]:
import random
def random_flip(x, p=0.5):
    if random.random() < p:
        return x.flip(-1)  # ランダムに画像を左右反転
    return x

In [ ]:
def relu(x):
    x = torch.where(x > 0, x, torch.zeros_like(x))
    return x


def softmax(x):
    x -= torch.cat([x.max(axis=1, keepdim=True).values] * x.size()[1], dim=1)
    x_exp = torch.exp(x)
    return x_exp/torch.cat([x_exp.sum(dim=1, keepdim=True)] * x.size()[1], dim=1)



class Dense(nn.Module):  # nn.Moduleを継承する
    def __init__(self, in_dim, out_dim, function=lambda x: x):
        super().__init__()
        # He Initialization
        # in_dim: 入力の次元数、out_dim: 出力の次元数
        self.W = nn.Parameter(torch.tensor(np.random.uniform(
                        low=-np.sqrt(6/in_dim),
                        high=np.sqrt(6/in_dim),
                        size=(in_dim, out_dim)
                    ).astype('float32')))
        self.b = nn.Parameter(torch.tensor(np.zeros([out_dim]).astype('float32')))
        self.function = function

    def forward(self, x):  # forwardをoverride
        return self.function(torch.matmul(x, self.W) + self.b)



class MLP(nn.Module):  # nn.Moduleを継承する
    def __init__(self, in_dim, hid_dim, out_dim):  # __init__をoverride
        super(MLP, self).__init__()
        self.linear1 = Dense(in_dim, hid_dim, function=relu)
        self.linear2 = Dense(hid_dim, out_dim, function=softmax)

    def forward(self, x):  # forwardをoverride
        x = self.linear1(x)
        x = self.linear2(x)
        return x

in_dim = 784
hid_dim = 2500
out_dim = 10
lr = 0.00008
n_epochs = 100


mlp = MLP(in_dim, hid_dim, out_dim).to(device)

optimizer = optim.Adam(mlp.parameters(), lr=lr)

In [ ]:
best_valid_acc = 0.0  # 最高の検証精度を保存するための変数を初期化
best_model_path = 'best_model.pth'  # 最良のモデルを保存するパス
for epoch in range(n_epochs):
        
    losses_train = []
    losses_valid = []
    train_num = 0
    train_true_num = 0
    valid_num = 0
    valid_true_num = 0

    mlp.train()  # 訓練時には勾配を計算するtrainモードにする
    for x, t in dataloader_train:
        #x = random_flip(x)  # データ拡張を適用
        true = t.tolist()

        t_hot = torch.eye(10)[t]  # 正解ラベルをone-hot vector化

        # テンソルをGPUに移動
        x = x.to(device)
        t_hot = t_hot.to(device)

        # 順伝播
        y = mlp.forward(x)

        # 誤差の計算(クロスエントロピー誤差関数)
        loss = -(t_hot*torch.log(y)).sum(axis=1).mean()

        # 誤差の逆伝播
        optimizer.zero_grad()
        loss.backward()

        # パラメータの更新
        optimizer.step()

        # モデルの出力を予測値のスカラーに変換
        pred = y.argmax(1)

        losses_train.append(loss.tolist())

        acc = torch.where(t - pred.to("cpu") == 0, torch.ones_like(t), torch.zeros_like(t))
        train_num += acc.size()[0]
        train_true_num += acc.sum().item()

    mlp.eval()  # 評価時には勾配を計算しないevalモードにする
    for x, t in dataloader_valid:
        true = t.tolist()

        t_hot = torch.eye(10)[t]  # 正解ラベルをone-hot vector化

        # テンソルをGPUに移動
        x = x.to(device)
        t_hot = t_hot.to(device)

        # 順伝播
        y = mlp.forward(x)

        # 誤差の計算(クロスエントロピー誤差関数)
        loss = -(t_hot*torch.log(y)).sum(axis=1).mean()

        # モデルの出力を予測値のスカラーに変換
        pred = y.argmax(1)

        losses_valid.append(loss.tolist())

        acc = torch.where(t - pred.to("cpu") == 0, torch.ones_like(t), torch.zeros_like(t))
        valid_num += acc.size()[0]
        valid_true_num += acc.sum().item()
    # 検証ループの後で、検証精度を計算
    valid_acc = valid_true_num / valid_num

    # 現在の検証精度がこれまでの最高精度を上回った場合、モデルを保存
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
        #mlp.load_state_dict(torch.load(best_model_path))

        mlp.eval()
        t_pred = []
        for x in dataloader_test:
            x = x.to(device)
            y = mlp.forward(x)  
            pred = y.argmax(1).tolist()
            t_pred.extend(pred)

        submission = pd.Series(t_pred, name='label')
        submission.to_csv('drive/MyDrive/Colab Notebooks/DLBasics2023_colab/Lecture04/submission_pre.csv', header=True, index_label='id')
        print('EPOCH: {}, Train [Loss: {:.3f}, Accuracy: {:.3f}], Valid [Loss: {:.3f}, Accuracy: {:.5f}]'.format(
            epoch,
            np.mean(losses_train),
            train_true_num/train_num,
            np.mean(losses_valid),
            valid_true_num/valid_num
        ))

        t_pred = []

EPOCH: 0, Train [Loss: 0.546, Accuracy: 0.816], Valid [Loss: 0.431, Accuracy: 0.84910]
EPOCH: 1, Train [Loss: 0.403, Accuracy: 0.861], Valid [Loss: 0.369, Accuracy: 0.87230]
EPOCH: 2, Train [Loss: 0.362, Accuracy: 0.874], Valid [Loss: 0.357, Accuracy: 0.87380]
EPOCH: 3, Train [Loss: 0.337, Accuracy: 0.882], Valid [Loss: 0.342, Accuracy: 0.87740]
EPOCH: 4, Train [Loss: 0.314, Accuracy: 0.889], Valid [Loss: 0.335, Accuracy: 0.87980]
EPOCH: 6, Train [Loss: 0.283, Accuracy: 0.898], Valid [Loss: 0.325, Accuracy: 0.88080]
EPOCH: 7, Train [Loss: 0.270, Accuracy: 0.905], Valid [Loss: 0.299, Accuracy: 0.89410]
EPOCH: 13, Train [Loss: 0.208, Accuracy: 0.926], Valid [Loss: 0.280, Accuracy: 0.89790]
EPOCH: 16, Train [Loss: 0.185, Accuracy: 0.935], Valid [Loss: 0.274, Accuracy: 0.89880]
EPOCH: 17, Train [Loss: 0.178, Accuracy: 0.938], Valid [Loss: 0.279, Accuracy: 0.90040]
EPOCH: 21, Train [Loss: 0.154, Accuracy: 0.948], Valid [Loss: 0.278, Accuracy: 0.90150]
EPOCH: 29, Train [Loss: 0.113, Accuracy

In [ ]:
mlp.eval()
t_pred = []
for x in dataloader_test:
    x = x.to(device)
    y = mlp.forward(x)  
    pred = y.argmax(1).tolist()
    t_pred.extend(pred)

submission = pd.Series(t_pred, name='label')
submission.to_csv('drive/MyDrive/Colab Notebooks/DLBasics2023_colab/Lecture04/submission_pred.csv', header=True, index_label='id')